In [1]:
from scvi.dataset import EbiData, MouseAtlas, UnionDataset, AnnDatasetFromAnnData
from Eval_basis import *
import scanpy as sc
import pandas as pd
import scipy.sparse as sparse

[2019-08-21 17:51:12,007] INFO - scvi._settings | Added StreamHandler with custom formatter to 'scvi' logger.
/home/michael/anaconda3/envs/scvi/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [9]:
ebi_1 = EbiData("./data", 'E-MTAB-6946', result_file='raw')

/home/michael/anaconda3/envs/scvi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: DtypeWarning: Columns (8,20) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
[2019-08-21 18:02:49,778] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-21 18:02:49,783] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-21 18:02:52,748] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-21 18:02:55,663] INFO - scvi.dataset.dataset | Downsampled from 167276 to 167276 cells


In [10]:
ebi_2 = EbiData("./data", 'E-MTAB-7320', result_file='raw')

[2019-08-21 18:03:34,800] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-21 18:03:34,802] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-21 18:03:35,941] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-21 18:03:37,031] INFO - scvi.dataset.dataset | Downsampled from 62848 to 62848 cells


In [12]:
fpaths_and_fnames = {'data': "./data/mouse_atlas/cleaned_data_sparse.npz",
                     'cell': "./data/mouse_atlas/cell_annotation.csv",
                     'gene': "./data/mouse_atlas/gene_annotation.csv",
                     'pheno': "./data/mouse_atlas/phenotype_data.csv"}
mouse_atlas = MouseAtlas("./data/mouse_atlas",
                   fpaths_and_fnames,
                   True,
                   False)

In [2]:
conv = pd.read_csv("./data/gene_maps/hugo_mouse_genes-proteincoding.csv", header=0, index_col=0)
conv.index = conv.index.str.lower()

data_path = os.path.join(("./data"))
mouse_data_path = os.path.join(data_path, "mouse_data")
dsets = [] 
for file in os.listdir(f"{data_path}/mouse_data"):
#     if "droplet" in file:
    dset = sc.read_h5ad(os.path.join(mouse_data_path, file))
    dset.obs.rename(columns={"cell_ontology_class": "cell_types"}, inplace=True)
    
    dset = AnnDatasetFromAnnData(dset)
    
    gns_conved = conv.reindex(np.char.lower(dset.gene_names))["ensembl"]
    if not isinstance(dset.X, np.ndarray):
        X = dset.X.toarray()
    else:
        X = dset.X
    mask = ~gns_conved.isnull()
    
    dset.gene_names = gns_conved[mask].values.astype(str)
    dset.X = X[:, mask]
    dset.cell_types = np.array([ct.replace("ï", "i") for ct in dset.cell_types])
    
    dsets.append(dset)

[2019-08-21 17:51:16,785] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-08-21 17:51:17,323] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-21 17:51:17,324] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-21 17:51:17,537] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-21 17:51:17,769] INFO - scvi.dataset.dataset | Downsampled from 7916 to 7916 cells
[2019-08-21 17:51:18,758] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-21 17:51:19,768] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-08-21 17:51:20,070] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-21 17:51:20,071] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-21 17:51:20,151] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-21 17:51:20,236] IN

[2019-08-21 17:52:19,868] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-08-21 17:52:20,098] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-21 17:52:20,099] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-21 17:52:20,138] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-21 17:52:20,181] INFO - scvi.dataset.dataset | Downsampled from 1561 to 1561 cells
[2019-08-21 17:52:20,345] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-21 17:52:20,745] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-08-21 17:52:21,037] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-21 17:52:21,038] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-21 17:52:21,105] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-21 17:52:21,178] IN

[2019-08-21 17:52:50,400] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-21 17:52:50,710] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-21 17:52:50,956] INFO - scvi.dataset.dataset | Downsampled from 9734 to 9734 cells
[2019-08-21 17:52:52,323] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-21 17:52:54,603] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-21 17:52:54,605] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-21 17:52:54,980] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-21 17:52:55,374] INFO - scvi.dataset.dataset | Downsampled from 29322 to 29322 cells
[2019-08-21 17:53:00,601] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-21 17:53:04,798] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-08-21 17:53:05,266] INFO - scvi.data

In [5]:
mouse_muris_senis = UnionDataset("./data", 
                                 gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding", 
                                 low_memory=False)

In [6]:
mouse_muris_senis.join_datasets(data_source="memory", 
                                data_target="memory",
                                gene_datasets=dsets)
mouse_muris_senis.name = "Tabula Muris Senis"

Concatenating datasets: 100%|██████████| 38/38 [04:58<00:00,  6.47s/it]
[2019-08-21 17:59:23,590] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-08-21 17:59:29,529] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-21 17:59:29,539] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-08-21 17:59:29,543] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-21 17:59:29,575] INFO - scvi.dataset.dataset | Joined 38 datasets to one of shape 290905 x 22250.


In [25]:
dsets = None
mouse_muris_senis.cell_types = np.array([ct.replace("ï", "i") for ct in mouse_muris_senis.cell_types])

In [26]:
complete_mouse = UnionDataset("./data", 
                              gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding", 
                              low_memory=False)

In [27]:
complete_mouse.join_datasets(data_source="memory", 
                             data_target="hdf5", 
                             out_fname="mouse_data_all", 
                             gene_datasets=[mouse_muris_senis, ebi_1, ebi_2, mouse_atlas])

Writing dataset ('UnionDataset', 'Tabula Muris Senis') to hdf5 file.


Writing sparse matrix iteratively to file...: 100%|██████████| 291/291 [01:53<00:00,  2.68it/s]

Writing dataset ('EbiData', 'E-MTAB-6946') to hdf5 file.



Writing sparse matrix iteratively to file...: 100%|██████████| 168/168 [00:58<00:00,  3.31it/s]


Writing dataset ('EbiData', 'E-MTAB-7320') to hdf5 file.


Writing sparse matrix iteratively to file...: 100%|██████████| 63/63 [00:14<00:00,  4.84it/s]


Writing dataset ('MouseAtlas', None) to hdf5 file.


Writing sparse matrix iteratively to file...: 100%|██████████| 1332/1332 [16:51<00:00,  1.37it/s]


GeneExpressionDataset object (unpopulated)

In [16]:
mouse_muris_senis.cell_types

array(['B cell', 'Bergmann glial cell',
       'Brush cell of epithelium proper of large intestine',
       'CD4+ positive macrophage', 'CD8-positive, alpha-beta T cell',
       'Cd4+ macrophage', 'DN3 thymocyte', 'DN4 thymocyte',
       'HSC enriched fraction', 'Il6 expressing cells',
       'Kidney-new-24m-cluster', 'Kupffer cell', 'Langerhans cell',
       'MPP Fraction A', 'MPP Fraction B', 'MPP Fraction C',
       'Marrow Cd4 T cell', 'Marrow-unknown-progenitor', 'NK cell',
       'Schwann cell', 'T cell', 'aortic endothelial cell', 'astrocyte',
       'atrial myocyte', 'basal cell', 'basal cell of epidermis',
       'basal epithelial cell of tracheobronchial tree', 'basophil',
       'bladder cell', 'bladder urothelial cell', 'brain pericyte',
       'brush cell', 'cardiac neuron', 'chondrocyte',
       'ciliated columnar cell of tracheobronchial tree',
       'double negative T cell', 'early pro-B cell', 'endocardial cell',
       'endothelial cell', 'endothelial cell of coronar

In [30]:
complete_mouse = UnionDataset("./data", 
                              gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding", 
                              data_load_filename="mouse_data_all",
                              low_memory=True)

In [33]:
complete_mouse.gene_names

Index(['ensmusg00000000001', 'ensmusg00000000003', 'ensmusg00000000028',
       'ensmusg00000000037', 'ensmusg00000000049', 'ensmusg00000000056',
       'ensmusg00000000058', 'ensmusg00000000078', 'ensmusg00000000085',
       'ensmusg00000000088',
       ...
       'ensmusg00000118431', 'ensmusg00000118438', 'ensmusg00000118441',
       'ensmusg00000118454', 'ensmusg00000118455', 'ensmusg00000118459',
       'ensmusg00000118462', 'ensmusg00000118465', 'ensmusg00000118471',
       'ensmusg00000118483'],
      dtype='object', name='ensembl', length=22250)

In [32]:
n_epochs = 100
colors=None

print("Training VAE")

trainer = train_vae(complete_mouse, "./data", f"max_data_model", n_epochs=n_epochs)
# trainer_small = train_vae(data_small, "./data", f"small_{tissue}_data_portion", n_epochs=n_epochs)



Training VAE


TypeError: string operation on non-string array

In [ ]:
ebi_with_celltypes = EbiData("./data")

In [24]:
dot_size = (mpl.rcParams['lines.markersize'] ** 2.0)

posterior_big = plot_tsne(trainer, trainer.model, complete_mouse, f"./max_data_model",
                          colors=colors, s=dot_size, edgecolors='black')

# posterior_ebi_annotated = trainer.create_posterior(model, ebi_with_celltypes, indices=np.arange(len(dataset)))
posterior_ebi_annotated = plot_tsne(trainer_big, trainer_big.model, ebi_with_celltypes, f"./plots/small_{tissue}_data_portion_in_big",
#                                    colors=colors, s=dot_size, edgecolors='black')


[b'B cell',
 b'Bergmann glial cell',
 b'Brush cell of epithelium proper of large intestine',
 b'CD4+ positive macrophage',
 b'CD8-positive, alpha-beta T cell',
 b'Cd4+ macrophage',
 b'DN3 thymocyte',
 b'DN4 thymocyte',
 b'HSC enriched fraction',
 b'Il6 expressing cells',
 b'Kidney-new-24m-cluster',
 b'Kupffer cell',
 b'Langerhans cell',
 b'MPP Fraction A',
 b'MPP Fraction B',
 b'MPP Fraction C',
 b'Marrow Cd4 T cell',
 b'Marrow-unknown-progenitor',
 b'NK cell',
 b'Schwann cell',
 b'T cell',
 b'aortic endothelial cell',
 b'astrocyte',
 b'atrial myocyte',
 b'basal cell',
 b'basal cell of epidermis',
 b'basal epithelial cell of tracheobronchial tree',
 b'basophil',
 b'bladder cell',
 b'bladder urothelial cell',
 b'brain pericyte',
 b'brush cell',
 b'cardiac neuron',
 b'chondrocyte',
 b'ciliated columnar cell of tracheobronchial tree',
 b'double negative T cell',
 b'early pro-B cell',
 b'endocardial cell',
 b'endothelial cell',
 b'endothelial cell of coronary artery',
 b'endothelial cell o